In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pytorch-ignite

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
# Standard libraries
import os, sys, glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Local libraries
sys.path.insert(0, '/content/drive/MyDrive/Modelo_Italia/')
from utils.image_processor import ImageProcessor

In [ ]:
 def save_cams(image, cams, CATS, loc, title=None):
        """Shows the input class activation maps (CAMs)"""
        cam_arr = cams
        q = len(CATS) + 1
        columns = 3 if q > q else q
        rows = int(q / columns) + (1 if int(q % columns) > 0 else 0) \
            if q > columns else 1
        fig, axs = plt.subplots(rows, columns, figsize=(20, 20))
        for idx, axi in enumerate(axs.flat):
            if idx == 0:
                axi.imshow(image)
                axi.axis("off")
                axi.set_title("Original Image", fontsize=14)
            elif idx < q:
                plt_cam = cam_arr[idx - 1]

                if plt_cam.shape != image.shape[:-1]:
                    plt_cam =\
                        np.array(Image.fromarray(
                            plt_cam).resize(image.shape[:-1]))
                axi.imshow(image)
                axi.imshow(plt_cam, alpha=0.6, vmin=0, vmax=1, cmap="jet")
                axi.set_title(
                    f"Cat {idx} - {CATS[idx-1]} CAM", fontsize=14)
                axi.axis("off")
            else:
                axi.imshow(np.zeros((1, 1)))
                axi.axis("off")
        fig.suptitle(title, fontsize=24)
        fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=0)
        plt.savefig(loc,bbox_inches='tight',pad_inches=0)

In [ ]:
CATS = ["suspicius_site"]

In [ ]:
! mkdir architecture
! cp -r /content/drive/MyDrive/Modelo_Italia/architecture/* architecture

In [ ]:
STATE_DICT_PATH = "/content/drive/MyDrive/Modelo_Italia/weights/checkpoint.pth"
model = 'architecture.resnet50_fpn'

In [ ]:
ip = ImageProcessor(CATS, STATE_DICT_PATH, model=model)

In [ ]:
img_path = "/content/drive/MyDrive/Modelo_Italia/test_img/"
img_name = "test_image_google"
img_file = img_path + img_name + ".png"
iw = ip.execute_cams_pred(img_file)
print(iw.classification_scores, iw.predicted_categories)
#iw.show_global_cams()

[0.99993336] {'suspicius_site': 0.99993336}


In [ ]:
iw.save_classification_scores_to_file(img_path + img_name + "_scores")

In [ ]:
scores = np.load(img_path + img_name + "_scores.npy")
print(scores)

[0.99993336]


In [ ]:
save_cams(iw.image, iw.global_cams, CATS, img_path + img_name + "cams.png", title=None)

Output hidden; open in https://colab.research.google.com to view.